In [2]:
from torch_geometric.nn import TAGConv
from torch_geometric.nn import AGNNConv
from torch.nn import Sequential, Linear, ReLU
import torch.nn.functional as F
from torch.nn import LogSoftmax
import torch.nn as nn

In [3]:
class GNN_Example(nn.Module):
    """
      This class defines a PyTorch module that takes in a graph represented in the PyTorch Geometric Data format,
      and outputs a tensor of predictions for each node in the graph. The model consists of one or more TAGConv layers,
      which are a type of graph convolutional layer.

      Args:
          node_dim (int): The number of node inputs.
          edge_dim (int): The number of edge inputs.
          output_dim (int, optional): The number of outputs (default: 1).
          hidden_dim (int, optional): The number of hidden units in each GNN layer (default: 50).
          n_gnn_layers (int, optional): The number of GNN layers in the model (default: 1).
          K (int, optional): The number of hops in the neighbourhood for each GNN layer (default: 2).
          dropout_rate (float, optional): The dropout rate to be applied to the output of each GNN layer (default: 0).

      """
    def __init__(self, node_dim, edge_dim, output_dim=1, hidden_dim=50, n_gnn_layers=1, K=2, dropout_rate=0):
        super().__init__()
        self.node_dim = node_dim
        self.edge_dim = edge_dim
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_gnn_layers = n_gnn_layers
        self.K = K
        self.dropout_rate = dropout_rate
        self.dropout = nn.Dropout(self.dropout_rate, inplace=False)
        self.relu = nn.PReLU()
        self.softmax = LogSoftmax(dim = -1)

        self.convs = nn.ModuleList()

        if n_gnn_layers == 1:
            self.convs.append(TAGConv(node_dim, output_dim, K=K))
        else:
            self.convs.append(TAGConv(node_dim, hidden_dim, K=K))

            for l in range(n_gnn_layers-2):
                self.convs.append(TAGConv(hidden_dim, hidden_dim, K=K))

            self.convs.append(TAGConv(hidden_dim, output_dim, K=K))

    def forward(self, data):
        """Applies the GNN to the input graph.

          Args:
              data (Data): A PyTorch Geometric Data object representing the input graph.

          Returns:
              torch.Tensor: The output tensor of the GNN.

          """
        x = data.x
        edge_index = data.edge_index
        edge_attr = data.edge_attr

        for i in range(len(self.convs)-1):
            x = self.convs[i](x=x, edge_index=edge_index, edge_weight=edge_attr)
            x = self.dropout(x)
            x = self.relu(x)

        x = self.convs[-1](x=x, edge_index=edge_index, edge_weight=edge_attr)
        x = self.softmax(x)
        return x

    def reset_parameters(self):
        """
        Reset all learnable parameters in GCN layers and Batch Normalization
        Layers.
        """
        for conv in self.convs:
            conv.reset_parameters()
        # for bn in self.bns:
        #     bn.reset_parameters()